In [1]:
import pandas as pd
from tqdm import tqdm
from Graph import Graph

Question 1 - Data
In this question we are going to make three separated graphs and then making them weighted and merged

First, we simply load our three different datasets with using of `pandas`

In [2]:
#Loading our datasets via Pandas
col_names = ["user-u","user-v","timestamp"] #Column names
answers_questions = pd.read_csv("sx-stackoverflow-a2q.txt",sep=" ",names=col_names) #Loading a2q dataset
comments_answers = pd.read_csv("sx-stackoverflow-c2a.txt",sep=" ",names=col_names) #Loading c2a dataset
comments_questions = pd.read_csv("sx-stackoverflow-c2q.txt",sep=" ",names=col_names) #Loading c2q dataset

In [3]:
#Getting minimum and maximum timestamp of the these datasets
timestamp_aq = [answers_questions["timestamp"].max(),answers_questions["timestamp"].min()]
timestamp_ca = [comments_answers["timestamp"].max(),comments_answers["timestamp"].min()]
timestamp_cq = [comments_questions["timestamp"].max(),comments_questions["timestamp"].min()]
print(timestamp_aq)
print(timestamp_cq)
print(timestamp_ca)

[1457266693, 1217567877]
[1457273428, 1217633097]
[1457273420, 1217645529]


As we see,the minimum and maximum timestamps belong to the comments2questions dataset
Maximum date is: Sunday, March 6, 2016 2:10:20 PM
Minimnum date is: Saturday, August 2, 2008 2:52:09 AM

I choose the date of 2010/01/01 and 2010/03/01 to analyze
So our start_ts = 1262304000
And end_ts  = 1267401600

In [4]:
#Before that, we drop every data which the questionare answer it himself
temp_df  = answers_questions[answers_questions["user-u"] != answers_questions["user-v"]]
answers_questions = temp_df

In [5]:
#Now we are mask the dataframes again to drop anything outside of our desired timestamps
start_ts = 1262304000
end_ts = 1267401600
def FilterDataframeByTimestamp(df,start_ts,end_ts):
    return df[df["timestamp"].between(start_ts,end_ts)]

In [6]:
#Now using our above function to filter our datasets
#Filtering each dataset by start_ts and end_ts
answers_questions = FilterDataframeByTimestamp(answers_questions,start_ts,end_ts)
comments_answers = FilterDataframeByTimestamp(comments_answers,start_ts,end_ts)
comments_questions = FilterDataframeByTimestamp(comments_questions,start_ts,end_ts)

Before I get the weights, I merge these three dataframes together and then calculate the weights

In [7]:
#Making a list of our dataframes to drop the timestamp column and concatenating later

df_list = [answers_questions,comments_answers,comments_questions]
df_list = [df.drop("timestamp",axis=1) for df in df_list]

In [8]:
df = pd.concat(df_list) #Concatig our three dataframes

In [9]:
#I created a new column which just create a string based on users names
df["weight"] = df.apply(lambda row: "{}-{}".format(row["user-u"],row["user-v"]),axis=1)

In [10]:
#Counting unique values to get edges with their weights
edges = df["weight"].value_counts().to_dict()

In [11]:
edges #just for demonstration

{'187606-187606': 609,
 '22656-22656': 462,
 '157882-157882': 434,
 '20862-20862': 415,
 '203907-203907': 360,
 '54680-54680': 339,
 '61974-61974': 311,
 '34397-34397': 295,
 '139459-139459': 282,
 '17034-17034': 280,
 '38360-38360': 269,
 '206367-206367': 261,
 '13249-13249': 260,
 '95810-95810': 259,
 '234815-234815': 258,
 '70604-70604': 238,
 '152083-152083': 230,
 '65358-65358': 222,
 '224671-224671': 213,
 '10661-10661': 209,
 '138475-138475': 208,
 '45914-45914': 199,
 '226621-226621': 193,
 '233522-233522': 191,
 '53114-53114': 182,
 '89771-89771': 178,
 '113794-113794': 175,
 '126014-126014': 173,
 '23354-23354': 167,
 '179910-179910': 163,
 '6309-6309': 157,
 '18936-18936': 154,
 '217332-217332': 152,
 '222908-222908': 152,
 '76337-76337': 151,
 '55159-55159': 151,
 '131527-131527': 151,
 '172211-172211': 151,
 '33708-33708': 150,
 '87234-87234': 148,
 '128165-128165': 147,
 '224346-224346': 147,
 '213474-213474': 143,
 '18393-18393': 140,
 '215966-215966': 138,
 '12950-12950

In [12]:
#Finally, I create the graph
edges_size = len(edges) #Size of edges dict
graph = Graph(edges_size) #Graph
#Now we add our edges to our graph
for n in tqdm(edges.items()):
    e = n[0].split("-")
    graph.AddEdge(e[0],e[1],int(n[1]))

100%|██████████| 379152/379152 [00:01<00:00, 377060.07it/s]


In [13]:
#Writing the final graph to a file
#It will writes two files, one is graph structure, second one it's weights of the graph structure
graph.WriteGraphToFile("graph")

Graph is saved


In [14]:
print(graph.GetGraph()) #Showing our graphs structure
graph.weights  #Showing our graph weights

[[187606 187606]
 [ 22656  22656]
 [157882 157882]
 ...
 [270302  81800]
 [ 13813  32491]
 [ 52249   8088]]


array([609., 462., 434., ...,   1.,   1.,   1.])